### Description: For arbitrary number of tractors, this set of functions calculates the ratio between field work and  transportation to the fields

### Environment set up:

#### 1. OS: Ubuntu 20.04 LTS, 
#### 2. physical resources: CPU: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz 16gb memory
#### 3. install miniconda: wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
#### 4. create python 3.8 environment: conda create -n telemetryEnv38 python=3.8
#### 5. install core libraries:
    - conda install -c conda-forge geopy
    - conda install -c conda-forge urllib3
    - conda install pandas
    
#### 6. activate environment: conda activate telemetryEnv38
#### 7. run jupyter: jupyter notebook

In [ ]:
import re
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt


def distance(lat1, lat2, lon1, lon2):
    """
    Approximates distance between two points by Haversine formula.
    Source: 
    https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
        
        params: lat1, lat2, lon1, lon2 => floats
        
        returns: distance_meters => float
    """ 
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers.
    r = 6371
      
    # calculate the result in meters
    return(c * r)*1000


def machines_resampled_dfs(data, freq, columns, write_resampled_dfs):
    """
    Performs mean-resampling of dataframe for each machines' Serial Number, 
    based on given frequency and columns list. If specified, results are saved 
    under "datasets/resampled" directory and such structure is needed if this parameter
    is used.Returns a list of stacked dataframes.
    
        param: data => pd.DataFrame
        param: minute (Min) frequency => str
        param: columns => list
        param: write_dfs => boolean
        
        returns: all_df => list
    """
    try:
        machines = data.SerialNumber.unique()
    
    except ValueError as e:
        
        print(e)
    
    all_dfs = []
    
    for m in range(len(machines)):
    
        # iterate over machines, select set of variables and sort by time
        df = data[data.SerialNumber == machines[m]]        
        df = df[columns]
        df = df.sort_values(by="DateTime").reset_index(drop=True)
    
        # initiate geolocator, resample ts to 2 mins intervals, aggregate by 
        # mean and drop missing observations when machines were in steady mode 
        # and filter out coordinates array
        geolocator = Nominatim(user_agent="geoapiExercises", timeout=5)
        gps_ts = df.resample(freq, on='DateTime').mean().dropna()
        coordinates = gps_ts.values[:,0:2]
    
        # create empty lists to populate
        fields = []
        cities = []
        road_field_ratio = []       
        distances = []
        
        for i in range(len(coordinates)):
        
            lng_i = round(coordinates[i][0], 6)
            lat_i = round(coordinates[i][1], 6)  
        
            # extract places, cities and fields from the address
            try:
                address = geolocator.reverse(str(lat_i)+","+str(lng_i)).raw
                location = address["place_id"] 
                city = address["address"]["city"]
                fields.append(location)
                cities.append(city)
            
            # handle exception in case for some coordinate location wasn't found 
            #(i.e. due to the rounding)
            except:
                address = {}
                address["address"] = {}
                fields.append(np.nan)
                cities.append(np.nan)

            # check if road is present in the address dictionary
            if 'road' in address["address"].keys():
            
                road_field_ratio.append(0)
            # if road is not present add field observation - this might cause overestimation
            else:
            
                road_field_ratio.append(1)
    
            # staring point
            if i == 0:
                distances.append(0)
            
            # calculate delta distance between 2 consecutive points
            else:
                # t-1 point
                lng_0 = coordinates[i-1][0] 
                lat_0 = coordinates[i-1][1]

                # apply distance function and append 
                dist = distance(lat_i, lat_0, lng_i, lng_0)
                distances.append(dist)
                
          # add information back to resampled dataframe        
        gps_ts["road_field_ratio"] = road_field_ratio
        gps_ts["fields"] = fields
        gps_ts["city"] = cities
        gps_ts["delta_distance"] = distances
        
        if write_resampled_dfs:
            gps_ts.to_csv("datasets/resampled/" + str(machines[m])+'_gps_ts.csv')
        
        all_dfs.append(gps_ts)
        
    return all_dfs


def calculate_field_ratio(data_path, freq, columns, write_results=True, write_resampled_dfs=True):
    
    try:
        data  = pd.read_csv(data_path)
        data['DateTime'] = pd.to_datetime(data['DateTime'])
    
    except ValueError as e:
        
        print("Path or timestamp might be incorrect or missing:", e)

    if freq.endswith('Min'):
        f = int(re.findall('\d+', freq)[0])
        
    else:
        raise Exception("Check frequency formating, i.e 5 minutes should be passed as '5Min'.")
        
    
    resampled_data = machines_resampled_dfs(data, freq, columns, write_resampled_dfs)
    # Calculate total hours (multiply by f (due to f mins resampling) and divide by 60 to get the hours)
    total_hours = [round(len(df)*f/60, 2) for df in resampled_data]
    # Calculate field hours
    field_hours = [round(len(df[df.road_field_ratio==1])*f/60, 2) for df in resampled_data]
    # Calculate field percentage
    field_percenages = np.array(field_hours)/np.array(total_hours)*100
    field_percenages = [round(i, 2) for i in field_percenages]
    field_percenages = [str(i) +'%' for i in field_percenages]
    
    # Create dataframe table depicting estimated absolute total/field operation hours and ratio

    field_ratio = pd.DataFrame(index=data.SerialNumber.unique(), 
                           columns = ["est. total operation hours", 
                                      "est. field hours", 
                                      "est. field/operation ratio"])
    field_ratio["est. total operation hours"] = total_hours
    field_ratio["est. field hours"] = field_hours
    field_ratio["est. field/operation ratio"] = field_percenages
    
    if write_results:
        field_ratio.to_csv("datasets/results_tables/field_ratio.csv")
    
    return field_ratio

In [ ]:
# Specify parameters
path = "claas-telematics.csv"
frequency = '8Min'
columns = ['DateTime', 'GpsLongitude', 'GpsLatitude', 
           'Engine_rpm', 'EngineLoad', 'FuelConsumption_l_h', 'SpeedGearbox_km_h', 'TempCoolant_C']



In [ ]:
# Run the estimation
results = calculate_field_ratio(data_path=path, freq=frequency, columns=columns)

In [ ]:
# Show results
results